In [1]:
from __future__ import annotations

%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path


from wufam.data.prepare_data import read_kf_data
from wufam.dataset import Dataset

PATH = Path("../data/kf_data")
START = "1970-01-01"
END = "2024-12-31"
DATASET = Dataset.BM_6_M
FACTORS_DATASET = Dataset.FACTORS_M
WEIGHTING = "value_weighted"
FACTOR_ANNUALIZE = 12

## 1.1

In [3]:
portfolios_total_r, portfolios_xs_r, factors_df, rf = read_kf_data(
    portfolios_filename=PATH / DATASET,
    factors_filename=PATH / FACTORS_DATASET,
    start_date=START,
    end_date=END,
    weighting=WEIGHTING,
)

In [4]:
assert (
    portfolios_total_r.shape[0]
    == portfolios_xs_r.shape[0]
    == factors_df.shape[0]
    == rf.shape[0]
)

## 1.4

In [5]:
from wufam.ap.uncond_factor_model import UncondFactorModel

capm = UncondFactorModel()
capm.fit(
    test_assets_xs_r=portfolios_xs_r,
    factors_df=factors_df[["Mkt-RF"]],
)

In [6]:
capm.grs_stat, capm.p_value

(np.float64(6.629569676484499), np.float64(8.082971012380241e-07))

In [7]:
capm.rmse_score(
    test_assets_xs_r=portfolios_xs_r,
    factors=factors_df[["Mkt-RF"]],
)

np.float64(0.0019845935046515113)

In [8]:
capm.r2_score(
    test_assets_xs_r=portfolios_xs_r,
    factors=factors_df[["Mkt-RF"]],
)

np.float64(-0.8179039451377383)

In [9]:
capm.r2_gls_score(
    test_assets_xs_r=portfolios_xs_r,
    factors=factors_df[["Mkt-RF"]],
)

np.float64(-0.5848124620555419)

Try monthly

In [10]:
ff_model = UncondFactorModel()
ff_model.fit(
    test_assets_xs_r=portfolios_xs_r,
    factors_df=factors_df,
)

In [11]:
ff_model.grs_stat, ff_model.p_value

(np.float64(5.26895705119617), np.float64(2.5726415393510304e-05))

In [12]:
ff_model.rmse_score(
    test_assets_xs_r=portfolios_xs_r,
    factors=factors_df,
)

np.float64(0.0010964596491440297)

In [13]:
ff_model.r2_score(
    test_assets_xs_r=portfolios_xs_r,
    factors=factors_df,
)

np.float64(0.44510206514402795)

In [14]:
ff_model.r2_gls_score(
    test_assets_xs_r=portfolios_xs_r,
    factors=factors_df,
)

np.float64(-0.16860504805673937)

## 1.5

In [15]:
split_date = portfolios_xs_r.index[len(portfolios_xs_r) // 2]

In [16]:
capm.fit(
    test_assets_xs_r=portfolios_xs_r.loc[:split_date],
    factors_df=factors_df[["Mkt-RF"]].loc[:split_date],
)

In [17]:
capm.rmse_score(
    test_assets_xs_r=portfolios_xs_r.loc[split_date:],
    factors=factors_df[["Mkt-RF"]].loc[split_date:],
)

np.float64(0.0014909569646982166)

In [18]:
capm.r2_score(
    test_assets_xs_r=portfolios_xs_r.loc[split_date:],
    factors=factors_df[["Mkt-RF"]].loc[split_date:],
)

np.float64(-1.1595217020999948)

In [19]:
capm.r2_gls_score(
    test_assets_xs_r=portfolios_xs_r.loc[split_date:],
    factors=factors_df[["Mkt-RF"]].loc[split_date:],
)

np.float64(-0.3658270839465645)

In [20]:
ff_model.fit(
    test_assets_xs_r=portfolios_xs_r.loc[:split_date],
    factors_df=factors_df.loc[:split_date],
)

In [21]:
ff_model.rmse_score(
    test_assets_xs_r=portfolios_xs_r.loc[split_date:],
    factors=factors_df.loc[split_date:],
)

np.float64(0.0010301585243249142)

In [22]:
ff_model.r2_score(
    test_assets_xs_r=portfolios_xs_r.loc[split_date:],
    factors=factors_df.loc[split_date:],
)

np.float64(-0.030944955328074064)

In [23]:
ff_model.r2_gls_score(
    test_assets_xs_r=portfolios_xs_r.loc[split_date:],
    factors=factors_df.loc[split_date:],
)

np.float64(-0.24602739512673666)